# Benchmark for simple ops

**Candidates**

* Numpy
* Numba
* Pythran

## Loads

In [1]:
%load_ext pythran.magic

In [2]:
import numpy as np
from numba import jit

## 1. Basic Summation

### 1-1. Python

In [3]:
def py_sum(n):
    s = 0
    for i in range(n):
        s += i
    return s

In [4]:
%timeit py_sum(10000000)

378 ms ± 3.87 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


### 1-2. Pythran

#### 1-2-1. Just Pythran

In [5]:
%%pythran -DUSE_XSIMD -march=native
#pythran export pyth_sum(int)
def pyth_sum(n):
    s = 0
    for i in range(n):
        s += i
    return s

In [6]:
%timeit pyth_sum(10000000)

798 µs ± 750 ns per loop (mean ± std. dev. of 7 runs, 1000 loops each)


#### 1-2-2. Pythran with numpy

In [7]:
%%pythran -DUSE_XSIMD -march=native
#pythran export pyth_np_sum(int)
import numpy as np
def pyth_np_sum(n):
    return np.sum(np.arange(0, n, 1))

In [8]:
%timeit pyth_np_sum(10000000)

607 µs ± 2.52 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


#### 1-2-3. Pythran with openmp

In [9]:
%%pythran -DUSE_XSIMD -fopenmp -march=native
#pythran export pyth_omp_sum(int)
def pyth_omp_sum(n):
    s = 0
    #omp parallel for reduction(+:s)
    for i in range(n):
        s += i
    return s

In [10]:
%timeit pyth_omp_sum(10000000)

172 µs ± 5.61 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)


### 1-3. Numba

In [11]:
@jit(nopython=True, fastmath=True)
def numba_sum(n):
    s = 0
    for i in range(n):
        s += i
    return s

In [12]:
%timeit numba_sum(10000000)

148 ns ± 0.784 ns per loop (mean ± std. dev. of 7 runs, 10000000 loops each)


## 2. Matrix Multiplication

### 2-1. Numpy

In [13]:
def np_matmul(a, b):
    c = a @ b
    return c[0,0]

In [14]:
a = np.random.rand(1000, 1000)
b = np.random.rand(1000, 1000)

In [15]:
%timeit np_matmul(a, b)

11.2 ms ± 408 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


### 2-2. Numba

In [16]:
@jit(nopython=True, fastmath=True)
def numba_matmul(a, b):
    c = a @ b
    return c[0,0]

In [17]:
%timeit numba_matmul(a, b)

11.5 ms ± 973 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)


### 2-3. Pythran

In [18]:
%%pythran -DUSE_XSIMD -fopenmp -march=native
#pythran export pyth_matmul(float64[:,:], float64[:,:])
import numpy as np
def pyth_matmul(a, b):
    c = a @ b
    return c[0,0]

In [19]:
%timeit pyth_matmul(a, b)

12.8 ms ± 956 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
